In [8]:
#import dependencies
import os
import json
from dotenv import load_dotenv
from supabase import create_client, Client
import requests

In [ ]:
#Initialize supabase connection

load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
GOOGLE_GEOCODE_KEY = os.getenv('GOOGLE_GEOCODE_API')

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Supabase client initialized successfully!")

Supabase client initialized successfully!


In [ ]:
#Helper function to read dataset
def getData(filePath):
    with open(filePath, 'r') as f:
        data = json.load(f)
        return data

In [12]:
#Helper function to get geocode with google API

def get_geocode(address_data):
    address = f"{address_data['premise']} {address_data['street']}, {address_data['locality']}, {address_data['administrative_area']}"
    uri = f"https://maps.googleapis.com/maps/api/geocode/json?key={GOOGLE_GEOCODE_KEY}&address={address}"
    response = requests.get(uri)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None

In [ ]:
#Insert function into supabase

def insert_data(data):
    for entry in data:
        address_data = {
            'country': entry['address']['country'] or '',
            'administrative_area': entry['address']['administrative_area'] or '',
            'sub_administrative_area': entry['address']['sub_administrative_area'] or '',
            'locality': entry['address']['locality'] or '',
            'postal_code': entry['address']['postal_code'] or '',
            'street': entry['address']['street'] or '',
            'premise': entry['address']['premise'] or '',
            'sub_premise': entry['address']['sub_premise'] or '',
        }

        # geocode for the address
        lat, lng = get_geocode(address_data)
        address_data['latitude'] = lat
        address_data['longitude'] = lng

        address_response = supabase.table('address').upsert(
            [address_data],
            on_conflict="country,administrative_area,sub_administrative_area,locality,postal_code,street,premise,sub_premise"
        ).execute()
        
        if address_response.data:
            address_id = address_response.data[0]['id']
        else:
            print(f"Error upserting address data: {address_response}")
            continue 
        
        # Convert None to 0 for numeric entries
        price = entry['price'] if entry['price'] is not None else 0
        bedrooms = entry['bedrooms'] if entry['bedrooms'] is not None else 0
        bathrooms = entry['bathrooms'] if entry['bathrooms'] is not None else 0
        square_feet = entry['square_feet'] if entry['square_feet'] is not None else 0
        acre_lot = entry['acre_lot'] if entry['acre_lot'] is not None else 0
        
        property_listing_data = {
            'price': price,
            'bedrooms': bedrooms,
            'bathrooms': bathrooms,
            'square_feet': square_feet,
            'sale_status': entry['sale_status'],
            'acre_lot': acre_lot,
            'tour_available': entry['tour_available'],
            'image_source': entry['image_source'],
            'address_id': address_id
        }
        
        property_response = supabase.table('property_listings').upsert(
            [property_listing_data],
            on_conflict='address_id,price,sale_status'
        ).execute()
        
        if property_response.data:
            print(f"Successfully upserted property listing: {property_listing_data}")
        else:
            print(f"Error upserting property listing data: {property_response}")

In [ ]:
#load json files
brooklyn_data = getData('../scraper/json-dump/brooklyn-2025-03-15-formatted.json')
manhattan_data = getData('../scraper/json-dump/manhattan-2025-03-15-formatted.json')
queens_data = getData('../scraper/json-dump/queens-2025-03-15-formatted.json')
staten_island_data = getData('../scraper/json-dump/staten-island-2025-03-15-formatted.json')
bronx_data = getData('../scraper/json-dump-bronx-2025-03-15-formatted.json')

In [ ]:
insert_data(bronx_data)

In [ ]:
insert_data(manhattan_data)

In [ ]:
insert_data(brooklyn_data)

In [ ]:
insert_data(queens_data)

In [ ]:
insert_data(staten_island_data)